# Multicolumn Identifier Support for Graph functions
MADlib graph functions are designed to have a vertex and an edge table. Each vertex has an id in the vertex table and a pair of ids in the edge table define a directed edge. Starting with version 1.20.0, multiple columns can be used as the vertex identifier for select graph functions.

In [1]:
%load_ext sql

In [2]:
%sql postgresql://okislal@localhost:6600/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

u'Connected: okislal@madlib'

In [3]:
%sql select madlib.version();
#%sql select version();

 * postgresql://okislal@localhost:6600/madlib
1 rows affected.


version
"MADlib version: 1.21.0-dev, git revision: rel/v1.20.0-4-g195895cc, cmake configuration time: Thu Aug 18 13:37:23 UTC 2022, build type: RelWithDebInfo, build system: Darwin-20.6.0, C compiler: Clang, C++ compiler: Clang"


# 1.  Create vertex and edge tables

In [4]:
%%sql 
DROP TABLE IF EXISTS vertex_multicol, edge_multicol;
CREATE TABLE vertex_multicol(
    node_id_major BIGINT,
    node_id_minor BIGINT
);
CREATE TABLE edge_multicol(
    conn_src_major BIGINT,
    conn_dest_major BIGINT,
    user_id_major BIGINT,
    conn_src_minor BIGINT,
    conn_dest_minor BIGINT,
    user_id_minor BIGINT
);
INSERT INTO vertex_multicol VALUES
(0, 0),
(1, 1),
(2, 2),
(3, 3),
(4, 4),
(5, 5),
(6, 6);
INSERT INTO edge_multicol VALUES
(0, 1, 1, 0, 1, 1),
(0, 2, 1, 0, 2, 1),
(0, 4, 1, 0, 4, 1),
(1, 2, 1, 1, 2, 1),
(1, 3, 1, 1, 3, 1),
(2, 3, 1, 2, 3, 1),
(2, 5, 1, 2, 5, 1),
(2, 6, 1, 2, 6, 1),
(3, 0, 1, 3, 0, 1),
(4, 0, 1, 4, 0, 1),
(5, 6, 1, 5, 6, 1),
(6, 3, 1, 6, 3, 1),
(0, 1, 2, 0, 1, 2),
(0, 2, 2, 0, 2, 2),
(0, 4, 2, 0, 4, 2),
(1, 2, 2, 1, 2, 2),
(1, 3, 2, 1, 3, 2),
(2, 3, 2, 2, 3, 2),
(3, 0, 2, 3, 0, 2),
(4, 0, 2, 4, 0, 2),
(5, 6, 2, 5, 6, 2),
(6, 3, 2, 6, 3, 2);

SELECT * from edge_multicol ORDER BY conn_src_major;

 * postgresql://okislal@localhost:6600/madlib
Done.
Done.
Done.
7 rows affected.
22 rows affected.
22 rows affected.


conn_src_major,conn_dest_major,user_id_major,conn_src_minor,conn_dest_minor,user_id_minor
0,2,1,0,2,1
0,4,1,0,4,1
0,4,2,0,4,2
0,2,2,0,2,2
0,1,2,0,1,2
0,1,1,0,1,1
1,3,2,1,3,2
1,3,1,1,3,1
1,2,1,1,2,1
1,2,2,1,2,2


# 2. PageRank

In [5]:
%%sql

DROP TABLE IF EXISTS pagerank_multicol_out, pagerank_multicol_out_summary;
SELECT madlib.pagerank(
    'vertex_multicol',                                                              -- Vertex table
    '[node_id_major,node_id_minor]',                                                -- Vertex id column
    'edge_multicol',                                                                -- Edge table
    'src=[conn_src_major,conn_src_minor], dest=[conn_dest_major,conn_dest_minor]',  -- Comma delimted string of edge arguments
    'pagerank_multicol_out',                                                        -- Output table of PageRank
    NULL,                                                                           -- Default damping factor (0.85)
    NULL,                                                                           -- Default max iters (100)
    NULL,                                                                           -- Default Threshold
    'user_id_major,user_id_minor',                                                  -- Grouping Columns
    '{{2,2},{4,4}}');                                                               -- Personalization vertices
SELECT * FROM pagerank_multicol_out ORDER BY user_id_major,user_id_minor,pagerank DESC;

 * postgresql://okislal@localhost:6600/madlib
Done.
1 rows affected.
14 rows affected.


user_id_major,user_id_minor,id,pagerank
1,1,"[0L, 0L]",0.270635964386
1,1,"[2L, 2L]",0.184232398514
1,1,"[3L, 3L]",0.166801820206
1,1,"[4L, 4L]",0.151661035568
1,1,"[6L, 6L]",0.0965411872855
1,1,"[1L, 1L]",0.0766610355683
1,1,"[5L, 5L]",0.0521896024087
2,2,"[0L, 0L]",0.448826703441
2,2,"[3L, 3L]",0.325943770128
2,2,"[2L, 2L]",0.256179815391


Look at the summary table:

In [6]:
%%sql
SELECT * FROM pagerank_multicol_out_summary;

 * postgresql://okislal@localhost:6600/madlib
2 rows affected.


user_id_major,user_id_minor,__iterations__
2,2,45
1,1,41


# 3. Weakly Connected Components

In [7]:
%%sql
DROP TABLE IF EXISTS wcc_multicol_out, wcc_multicol_out_summary;
SELECT madlib.weakly_connected_components(
    'vertex_multicol',                                                              -- Vertex table
    '[node_id_major,node_id_minor]',                                                -- Vertex id column
    'edge_multicol',                                                                -- Edge table
    'src=[conn_src_major,conn_src_minor], dest=[conn_dest_major,conn_dest_minor]',  -- Comma delimted string of edge arguments
    'wcc_multicol_out',                                                             -- Output table of weakly connected components
    'user_id_major,user_id_minor');                                                 -- Grouping column name
SELECT * FROM wcc_multicol_out ORDER BY user_id_major, user_id_minor, component_id, id;

 * postgresql://okislal@localhost:6600/madlib
Done.
1 rows affected.
14 rows affected.


id,component_id,user_id_major,user_id_minor
"[0L, 0L]",3,1,1
"[1L, 1L]",3,1,1
"[2L, 2L]",3,1,1
"[3L, 3L]",3,1,1
"[4L, 4L]",3,1,1
"[5L, 5L]",3,1,1
"[6L, 6L]",3,1,1
"[0L, 0L]",3,2,2
"[1L, 1L]",3,2,2
"[2L, 2L]",3,2,2


# 4. WCC Helper Functions

In [10]:
%%sql
DROP TABLE IF EXISTS vc_table;
SELECT madlib.graph_wcc_vertex_check(
     'wcc_multicol_out',      -- WCC's output table
     '{{4,4},{5,5}}', -- Pair of vertex IDs
     'vc_table');         -- output table
SELECT * FROM vc_table;

 * postgresql://okislal@localhost:6600/madlib
Done.
1 rows affected.
2 rows affected.


user_id_major,user_id_minor,component_id
2,2,3
1,1,3


In [14]:
%%sql
DROP TABLE IF EXISTS reach_table;
SELECT madlib.graph_wcc_reachable_vertices(
     'wcc_multicol_out',    -- WCC's output table
     '{0,0}'::BIGINT[], -- source vertex
     'reach_table');    -- output table
SELECT * FROM reach_table ORDER BY user_id_major, user_id_minor;

 * postgresql://okislal@localhost:6600/madlib
Done.
1 rows affected.
12 rows affected.


user_id_major,user_id_minor,component_id,dest
1,1,3,"[5L, 5L]"
1,1,3,"[4L, 4L]"
1,1,3,"[2L, 2L]"
1,1,3,"[6L, 6L]"
1,1,3,"[3L, 3L]"
1,1,3,"[1L, 1L]"
2,2,3,"[2L, 2L]"
2,2,3,"[1L, 1L]"
2,2,3,"[6L, 6L]"
2,2,3,"[4L, 4L]"
